# Esercizio 5

Considerare un file `GTF` (Gene Transfer Format) che annota un set di geni e il file `FASTA` della genomica di riferimento, e produrre:

- le sequenze in formato `FASTA` dei trascritti full-length annotati o delle coding sequences (CDS) annotate
- il set dei geni (identificati tramite il loro HUGO name) relativi alle sequenze prodotte

L'*header* `FASTA` di ogni sequenza prodotta (trascritto o CDS) deve contenere:
- lo HUGO name del gene di riferimento
- l’identificatore del trascritto di riferimento
- la lunghezza della sequenza prodotta
- il tipo di sequenza (trascritto o CDS)
- lo strand del gene
    
Esempio di *header* per i trascritti:
         
    >ARHGAP4; U52112.4-003; len=3235 type=transcript; strand=-

Esempio di *header* per le CDS:

    >AVPR2; U52112.2-003; len=642; type=cds; strand=+
   
***

Parametri in input:

- nome del file in formato `GTF`
- nome del file della *genomic reference* in formato `FASTA`
- nome della *feature* `GTF` che compone la sequenza da ricostruire: `exon` per i trascritti full-length e `CDS` per le coding sequences

***

Requisiti:

- deve essere definita una funzione `format_fasta()` che prenda come argomenti un'intestazione `FASTA` e una sequenza, e restituisca la sequenza in formato FASTA separata in righe di 80 caratteri.

- deve essere definita una funzione `reverse_complement()` che prenda come argomento una sequenza nucleotidica e ne restituisca il reverse&complement.

- deve essere definita una funzione `compose_feature()` che prenda come argomenti una lista di *features* dello stesso tipo (in cui ogni *feature* è una tupla *(start, end)*), la genomica di riferimento, lo strand delle *features* rispetto alla genomica di riferimento (cioé lo strand del gene a cui le *features* appartengono), ed effettui la concatenazione delle sequenze delle *features*, ed eventualmente il reverse&complement se lo strand del gene è `-` (strand minus).

**NOTA BENE**: gli attributi del nono campo del file `GTF` non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `sequence_fasta_list`: lista delle sequenze ricostruite (trascritti full-length o coding sequences) in formato `FASTA`
- `gene_set`: set dei geni coinvolti nelle sequenze ricostruite

***

## Soluzione

Importare il modulo `re` per usare le espressioni regolari.

In [1]:
import re

### Definizione della funzione `format_fasta()`

La funzione prende come argomento una stringa contenente un *header* `FASTA` e una sequenza (nucleotidica o proteica) e restituisce la sequenza in formato `FASTA`.

In [2]:
def format_fasta(header, sequence):
    return header + '\n' + '\n'.join(re.findall('\w{1,80}', sequence))

**NOTA BENE**: supporre che l'*header* in input alla funzione abbia già il simbolo `>` all'inizio, ma non il simbolo `\n` alla fine.

### Definizione della funzione `reverse_complement()`

La funzione prende come argomento una stringa contenente una sequenza nucleotidica e restituisce la versione *reverse and complement* della sequenza.

**NOTA BENE**: fare in modo che la funzione restituisca la sequenza in maiuscolo.

### Definizione della funzione `compose_feature()`

La funzione prende come argomenti una lista di *features* dello stesso tipo (in cui ogni *feature* è una tupla *(start, end)*), la genomica di riferimento, lo strand delle *features* rispetto alla genomica di riferimento (cioé lo strand del gene a cui le *features* appartengono), ed effettua la concatenazione delle sequenze delle *features*, ed eventualmente il reverse&complement se lo strand del gene è `-`.

**NOTA BENE**: la concatenazione delle *features* deve sempre avvenire per coordinate crescenti delle *features*.

### Parametri in input

In [5]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'
feature_name = 'exon'

### Lettura del file `FASTA` della genomica di riferimento

Lettura del file della genomica di riferimento nella lista di righe `reference_file_rows`

In [7]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

Concatenazione delle righe della lista `reference_file_rows` che contengono la sequenza nucleotidica (dopo avere eliminato il simbolo di *newline* finale).

In [9]:
genomic_reference

'...'

### Lettura dei *record* del file `GTF`

Lettura dei *record* del file `GTF` nella lista di righe `gtf_file_rows`

In [11]:
gtf_file_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52

### Filtraggio dei *record* `GTF` che occorrono per ricostruire le sequenze del tipo scelto

Eliminare dalla lista `gtf_file_rows` i *record* `GTF` che non corrispondono al tipo di *feature* che compone la sequenza che si è scelto di ricostruire, cioé per i quali il terzo campo non è uguale al valore della variabile `feature_name` (`exon` se si è scelto di ricostruire i trascritti full-length e `CDS` se si è scelto di ricostruire le coding sequences).

In [13]:
gtf_file_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64385\t64459\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t79484\t79511\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72761\t72965\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72521\t72683\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72253\t72379\t.\t-\t.\ttranscript_id

### Costruzione del dizionario degli *strand* e del set dei geni annotati

A partire dalla lista `gtf_file_rows`, costruire:
- il dizionario `strand_dict` degli *strand* dei geni:
    - *chiave*: HUGO name del gene
    - *valore*: strand del gene (`+` o `-`)
    
- il set `gene_set` dei geni annotati relativamente al tipo di sequenza che si vuole ricostruire

**NOTA BENE**: il valore dello *strand* (settimo campo del *record* `GTF`) è costante lungo tutti i *record* relativi allo stesso gene.

Inizializzazione del dizionario vuoto.

Attraversare la lista `gtf_file_rows` e riempire il dizionario `strand_dict`.

In [16]:
strand_dict

{'ARHGAP4': '-', 'ATP6AP1': '+', 'AVPR2': '+'}

Estrarre dal dizionario `strand_dict` il set `gene_set` dei geni annotati.

In [18]:
gene_set

{'ARHGAP4', 'ATP6AP1', 'AVPR2'}

### Ricostruzione delle sequenze

Costruire:
- il dizionario `id_dict` degli ID dei trascritti:
    - *chiave*: HUGO name del gene
    - *valore*: set degli identificatori dei trascritti per cui è annotata il tipo di sequenza che si è scelto di ricostruire
- il dizionario `composition_dict`:
    - *chiave*: identificatore del trascritto
    - *valore*: lista delle *features* `GTF`, intese come tuple *(start, end)*, che compongono la sequenza da ricostruire

Inizializzare i dizionari vuoti.

Attraversare la lista `gtf_file_rows` e riempire i due dizionari.

In [21]:
id_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-002',
  'U52112.4-003',
  'U52112.4-004',
  'U52112.4-005',
  'U52112.4-006',
  'U52112.4-007',
  'U52112.4-008',
  'U52112.4-009',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-012',
  'U52112.4-013',
  'U52112.4-014',
  'U52112.4-015',
  'U52112.4-016',
  'U52112.4-017',
  'U52112.4-018',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-021',
  'U52112.4-022',
  'U52112.4-023',
  'U52112.4-024'},
 'ATP6AP1': {'XX-FW83563B9.4-001',
  'XX-FW83563B9.4-002',
  'XX-FW83563B9.4-003',
  'XX-FW83563B9.4-004',
  'XX-FW83563B9.4-006'},
 'AVPR2': {'U52112.2-001', 'U52112.2-002', 'U52112.2-003'}}

In [22]:
composition_dict

{'U52112.4-005': [(71783, 71788),
  (70312, 70440),
  (69989, 70210),
  (64935, 65036),
  (64566, 64673),
  (64385, 64459)],
 'U52112.4-018': [(79484, 79511),
  (72761, 72965),
  (72521, 72683),
  (72253, 72379),
  (71896, 71965)],
 'U52112.4-014': [(77293, 77462),
  (72761, 72965),
  (72521, 72683),
  (72253, 72315),
  (71783, 71993),
  (70312, 70440),
  (69989, 70210),
  (64935, 65036),
  (64566, 64757),
  (64375, 64459),
  (64181, 64208),
  (63857, 63959),
  (62286, 62346),
  (62079, 62156),
  (61857, 61991),
  (61663, 61768),
  (61328, 61561),
  (61169, 61242),
  (60898, 61081),
  (60600, 60692),
  (60227, 60326),
  (58626, 59119)],
 'U52112.4-022': [(86040, 86155),
  (85533, 85631),
  (85099, 85157),
  (83695, 83740),
  (83472, 83587),
  (83227, 83271),
  (72761, 72965),
  (72521, 72683),
  (72253, 72315),
  (71783, 71965),
  (70312, 70440),
  (69989, 70210),
  (64935, 65036),
  (64367, 64757)],
 'U52112.4-021': [(72521, 72556), (72253, 72315), (71569, 71965)],
 'U52112.4-006': [(

A partire dai dizionari `id_dict`, `composition_dict` e `strand_dict`, costruire la lista `sequence_fasta_list` di tuple *(header, sequenza)* in cui il primo elemento è l'*header* `FASTA` e il secondo elemento è la sequenza ricostruita.

L'*header* deve essere del tipo:

    >ARHGAP4; U52112.4-003; len=3235; type=transcript; strand=-
    
se si è scelto di ricostruire i trascritti full-length, e:

    >ARHGAP4; U52112.4-005; len=642; type=cds; strand=-
    
se si è scelto di ricostruire le coding sequences (CDS).

In [24]:
sequence_fasta_list

[('>ARHGAP4; U52112.4-002; len=2817; type=transcript; strand=-',
  'CGTGGGAGCAGTGGGGTTCGACGGCGCGGCCGCGAGGCCGCCATGGCCGCTCACGGGAAGCTGCGGCGGGAGCGGGGGCTGCAGGCTGAGTATGAGACGCAAGTCAAAGAGATGCGCTGGCAGCTGAGCGAGCAGCTGCGCTGCCTGGAGCTGCAGGGCGAGCTGCGGCGGGAGTTGCTGCAGGAGCTGGCAGAGTTCATGCGGCGCCGCGCTGAGGTGGAGCTGGAATACTCCCGGGGCCTGGAAAAGCTGGCCGAGCGCTTCTCCAGCCGTGGAGGCCGCCTGGGGAGCAGCCGGGAGCACCAAAGCTTCCGGAAGGAGCCGTCCCTCCTGTCGCCCTTGCACTGCTGGGCGGTGCTGCTGCAGCACACGCGGCAGCAGAGCCGGGAGAGCGCGGCCCTGAGTGAGGTGCTGGCCGGGCCCCTGGCCCAGCGCCTGAGTCACATTGCAGAGGACGTGGGGCGCCTGGTCAAGAAGAGCAGGGATCTGGAGCAGCAGCTGCAGGATGAGCTCCTGGAGGTGGTCTCAGAGCTCCAGACGGCCAAGAAGACGTACCAGGCATATCACATGGAGAGCGTGAATGCCGAGGCCAAGCTCCGGGAGGCCGAGCGGCAGGAGGAGAAGCGGGCAGGCCGGAGTGTCCCCACCACCACCGCTGGTGCCACTGAGGCAGGGCCCCTCCGCAAGAGCTCCCTCAAGAAGGGAGGGAGGCTGGTGGAGAAGCGGCAGGCCAAGTTCATGGAGCACAAACTCAAGTGCACAAAGGCGCGCAACGAGTACCTGCTTAGCCTGGCTAGTGTCAACGCTGCTGTCAGTAACTACTACCTGCATGACGTCTTGGACCTCATGGACTGCTGTGACACAGGGTTCCACCTGGCCCTGGGGCAGGTGCTCCGGAGCTACACGGCCGCTGAGAGCCGCACCCAAGCCTC

Trasformare la lista di tuple `feature_fasta_list` in una lista di sequenze in formato `FASTA`.

In [26]:
for seq in sequence_fasta_list:
    print(seq)

>ARHGAP4; U52112.4-002; len=2817; type=transcript; strand=-
CGTGGGAGCAGTGGGGTTCGACGGCGCGGCCGCGAGGCCGCCATGGCCGCTCACGGGAAGCTGCGGCGGGAGCGGGGGCT
GCAGGCTGAGTATGAGACGCAAGTCAAAGAGATGCGCTGGCAGCTGAGCGAGCAGCTGCGCTGCCTGGAGCTGCAGGGCG
AGCTGCGGCGGGAGTTGCTGCAGGAGCTGGCAGAGTTCATGCGGCGCCGCGCTGAGGTGGAGCTGGAATACTCCCGGGGC
CTGGAAAAGCTGGCCGAGCGCTTCTCCAGCCGTGGAGGCCGCCTGGGGAGCAGCCGGGAGCACCAAAGCTTCCGGAAGGA
GCCGTCCCTCCTGTCGCCCTTGCACTGCTGGGCGGTGCTGCTGCAGCACACGCGGCAGCAGAGCCGGGAGAGCGCGGCCC
TGAGTGAGGTGCTGGCCGGGCCCCTGGCCCAGCGCCTGAGTCACATTGCAGAGGACGTGGGGCGCCTGGTCAAGAAGAGC
AGGGATCTGGAGCAGCAGCTGCAGGATGAGCTCCTGGAGGTGGTCTCAGAGCTCCAGACGGCCAAGAAGACGTACCAGGC
ATATCACATGGAGAGCGTGAATGCCGAGGCCAAGCTCCGGGAGGCCGAGCGGCAGGAGGAGAAGCGGGCAGGCCGGAGTG
TCCCCACCACCACCGCTGGTGCCACTGAGGCAGGGCCCCTCCGCAAGAGCTCCCTCAAGAAGGGAGGGAGGCTGGTGGAG
AAGCGGCAGGCCAAGTTCATGGAGCACAAACTCAAGTGCACAAAGGCGCGCAACGAGTACCTGCTTAGCCTGGCTAGTGT
CAACGCTGCTGTCAGTAACTACTACCTGCATGACGTCTTGGACCTCATGGACTGCTGTGACACAGGGTTCCACCTGGCCC
TGGGGCAGGTGCTCCGGAGCTACACGGCCGCTGAGAGCCGCACCCAAGC